<a href="https://colab.research.google.com/github/tanzilahmed0/CS-133/blob/main/Tanzil_Ahmed__JOption1_CS133_HO12_sqlite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###**Q1**.  Use Lucidchart to create an Entity Relationship Diagram (ERD) for the following tables representing a customer order tracking system:

Tables and fields:
1. product:
    - product_id: INTEGER (Primary key)
    - name: TEXT not null
    - price: REAL


2. customer:
    - customer_id: INTEGER (Primary key)
    - name: TEXT not null
    - email: TEXT not null


3. purchase_order:
    - order_id: INTEGER (Primary key)
    - customer_id: INTEGER (Foreign key)
    - date: TEXT not null ("YYYY-MM-DD")


4. order_item:
    - order_id: INTEGER (Foreign key)
    - product_id: INTEGER (Foreign key)
    - quantity: INTEGER
    


Export the ERD as PDF and submit it in Canvas.

###**Q2**. Create SQLite tables and load data
1. Here are the csv files for the data for the four tables:
   - product: https://raw.githubusercontent.com/csbfx/cs133/main/product.csv
   - customer: https://raw.githubusercontent.com/csbfx/cs133/main/customer.csv
   - order_item: https://raw.githubusercontent.com/csbfx/cs133/main/order_item.csv
   - purchase_order: https://raw.githubusercontent.com/csbfx/cs133/main/purchase_order.csv
2. In this notebook, create the database and save it in a file called `store.db`, and create the four tables as described above.
3. Load the data in the csv files into the corresponding table.
4. Commit so that the data loaded to the tables to officially written to the tables.
5. Execute a query SELECT * from each table to make sure the data are properly loaded.
6. Execute a query using SELECT statement that queries with JOIN tables to find the purchase date, the products and quantities that a particular customer has purchased.

In [ ]:
# 2.2 Create the database and save it in a file called store.db, and create the four tables as described above.
# Your code here . . .
import pandas as pd
import sqlite3
from pathlib import Path



conn = sqlite3.connect('store.db')
c = conn.cursor()

product_table = '''CREATE TABLE IF NOT EXISTS product (
    product_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    price REAL
    )'''

customer_table = '''CREATE TABLE IF NOT EXISTS customer (
    customer_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    email TEXT NOT NULL
    )'''

purchase_order_table = '''CREATE TABLE IF NOT EXISTS purchase_order (
    order_id INTEGER PRIMARY KEY,
    customer_id INTEGER,
    date TEXT NOT NULL,
    FOREIGN KEY (customer_id) REFERENCES customer(customer_id)
    )'''
order_item_table = '''CREATE TABLE IF NOT EXISTS order_item (
    order_id INTEGER,
    product_id INTEGER,
    quantity INTEGER,
    FOREIGN KEY (order_id) REFERENCES purchase_order(order_id),
    FOREIGN KEY (product_id) REFERENCES product(product_id)
    )'''



c.execute(product_table)
c.execute(customer_table)
c.execute(purchase_order_table)
c.execute(order_item_table)



In [ ]:
# 2.3 Load the data in the csv files into the corresponding table.
# Your code here . . .
product = pd.read_csv('https://raw.githubusercontent.com/csbfx/cs133/main/product.csv')
customer = pd.read_csv('https://raw.githubusercontent.com/csbfx/cs133/main/customer.csv')
purchase_order = pd.read_csv('https://raw.githubusercontent.com/csbfx/cs133/main/order_item.csv')
order_item = pd.read_csv('https://raw.githubusercontent.com/csbfx/cs133/main/purchase_order.csv')


product.to_sql('product', conn, if_exists='replace', index=False)
customer.to_sql('customer', conn, if_exists='replace', index=False)
purchase_order.to_sql('purchase_order', conn, if_exists='replace', index=False)
order_item.to_sql('order_item', conn, if_exists='replace', index=False)

3

In [ ]:
# 2.4 Commit so that the data loaded to the tables to officially written to the tables.
# Your code here . . .
conn.commit()

In [ ]:
# 2.5 Execute a query SELECT * from each table to make sure the data are properly loaded.
# Your code here . . .
c.execute('SELECT * FROM product')
p_results = c.fetchall()

c.execute('SELECT * FROM customer')
c_results = c.fetchall()

c.execute('SELECT * FROM purchase_order')
p_o_results = c.fetchall()

c.execute('SELECT * FROM order_item')
o_i_results = c.fetchall()


print(p_results)
print(c_results)
print(p_o_results)
print(o_i_results)

[(0, 'bicycle', 400), (1, 'helmet', 45), (2, 'gloves', 23), (3, 'chain', 48)]
[(0, 'Wendy Lee', 'wlee@bike.com'), (1, 'Jason Brown', 'jb@speed.com'), (2, 'Harry Potter', 'hp@hogwarts.edu'), (3, 'Godric Gryffindor', 'gg@hogwards.edu')]
[(0, 0, 1), (0, 1, 1), (1, 2, 2), (1, 3, 1)]
[(0, 0, '2020-10-19'), (1, 0, '2020-10-20'), (2, 1, '2020-10-20')]


In [ ]:
# 2.6 Execute a query using SELECT statement that queries with JOIN tables to find the purchase date,
#    the products and quantities that a particular customer has purchased.
# Your code here . . .


query = """
SELECT
    customer.name AS customer_name,
    product.name AS product_name,
    order_item.date AS purchase_date,
    purchase_order.quantity AS quantity
FROM
    customer
JOIN
    order_item ON customer.customer_id = order_item.customer_id
JOIN
    purchase_order ON order_item.order_id = purchase_order.order_id
JOIN
    product ON purchase_order.product_id = product.product_id
ORDER BY
    customer_name, purchase_date;
"""


df_result = pd.read_sql_query(query, conn)


conn.close()


df_result.head()

,customer_name,product_name,purchase_date,quantity
0,Wendy Lee,bicycle,2020-10-19,1
1,Wendy Lee,helmet,2020-10-19,1
2,Wendy Lee,gloves,2020-10-20,2
3,Wendy Lee,chain,2020-10-20,1
